# Data Preparation

In [ ]:
import skimage as ski
import numpy as np

In [ ]:
import sys
sys.path.append('../../DataPreparation')

import sys
sys.path.append('../../FeatureExtraction/SIFT')

In [ ]:
from DataPreparation import preprocess_and_save_images

# Preprocess and save images
preprocess_and_save_images(start_index = 0, batch_size = 10)

In [ ]:
from DataPreparation import load_images

# Load preprocessed images
x_data, y_data = load_images(start_index = 0, batch_size = 10)

# Feature Extraction

In [ ]:
from SIFT import compute_sift_descriptors_to_train_Kmeans

sift_descriptors = compute_sift_descriptors_to_train_Kmeans(x_data)

In [ ]:
from SIFT import train_kmeans

kmeans = train_kmeans(sift_descriptors, "MiniBatchKMeans")

In [ ]:
from SIFT import compute_sift_descriptors_to_train_SVM

sift_descriptors_svm = compute_sift_descriptors_to_train_SVM(x_data)

In [ ]:
from SIFT import compute_histograms

feature_vectors = compute_histograms(kmeans, sift_descriptors_svm)

# Model Training

## Prepare Data for SVM
We then scale our feature vectors using the StandardScaler from sklearn. This is a good practice before using SVM.

In [ ]:
from SVM import prepare_data_for_svm

scaled_feature_vectors = prepare_data_for_svm(feature_vectors)

## Train SVM
We then train our SVM classifier using the scaled feature vectors. We're using a Radial Basis Function (RBF) kernel.

In [ ]:
from SVM import train_svm

clf = train_svm(scaled_feature_vectors, y_data)

## Test SVM
Finally, we test our trained SVM model on the test data and print the score.

In [ ]:
from DataPreparation import load_images
from SVM import test_svm

# Load preprocessed images
x_test, y_test = load_images(start_index = 500, batch_size = 50)

# Compute the feature vectors for the test set
feature_vectors_test = compute_histograms(kmeans, compute_sift_descriptors_to_train_SVM(x_test))

# Test the SVM
score = test_svm(clf, feature_vectors_test, y_test)

print("Score: ", score)